In [1]:
from tinygrad.densetensor import DenseTensor
from tinygrad.sparsetensor import SparseTensor
import numpy as np

%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
x_init = np.random.randn(2,6).astype(np.float32)
x2_init = np.random.randn(3).astype(np.float32)
U_init = np.random.randn(3,3).astype(np.float32)
V_init = np.random.randn(3,3).astype(np.float32)
W_init = np.random.randn(6,3).astype(np.float32)
m_init = np.random.randn(1,3).astype(np.float32)

In [3]:
x = DenseTensor(x_init)
W = DenseTensor(W_init)
m = DenseTensor(m_init)
out = x.dot(W).relu()
out = out.logsoftmax()
out = out.mul(m).add(m).sum()
out.backward()

out.cpu().data, x

(array([-1.8180904], dtype=float32),
 <DenseTensor <GPUBuffer with shape (2, 6)> with grad <GPUBuffer with shape (2, 6)>>)

x2 = DenseTensor(x2_init)#.gpu()
W = SparseTensor(W_init)
out = W.dot(x2).relu().sum()

out.backward()

out.cpu().data, x

In [4]:
import numpy as np
import pyopencl as cl

mf = cl.mem_flags

In [5]:
dim1 = 4
dim2 = 8
dim3 = 3
bs = dim3

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(dim2,dim3).astype(np.float32)

a.shape, b.shape

((4, 8), (8, 3))

In [6]:
def fill_sparse(mat, sparsity=0.1):
    indices = np.array(range(mat.shape[1]))
    nrows = int(mat.shape[1]*sparsity)
    for row in range(mat.shape[0]):
        lim = nrows #+ int(np.random.random()*3)
        mat[row][np.random.permutation(indices)[:lim]] = np.random.random(lim)
    return mat

a = fill_sparse(a, 1).astype(np.float32)
#b = fill_sparse(b, sparsity)

In [7]:
a

array([[0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 ],
       [0.6204036 , 0.21273285, 0.59999454, 0.7183614 , 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.8756672 , 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.7515835 , 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.6807204 , 0.61051154]], dtype=float32)

In [8]:
b

array([[0.15043414, 0.6196845 , 0.35063207],
       [0.4796516 , 0.5857214 , 0.90562654],
       [0.08651726, 0.82600874, 0.6354987 ],
       [0.74133885, 0.2394775 , 0.5392108 ],
       [0.777668  , 0.3423175 , 0.10126331],
       [0.2247686 , 0.01690849, 0.9917596 ],
       [0.8016237 , 0.904124  , 0.01441843],
       [0.88494974, 0.20410454, 0.5339545 ]], dtype=float32)

In [9]:
x2_init.T

array([-0.16161858, -0.3260136 ,  1.5354537 ], dtype=float32)

In [10]:
mult = a.dot(b)
mult.shape

(4, 3)

In [11]:
mult.shape

(4, 3)

In [12]:
def to_data(mat):
    all_rows = []
    all_idxs = []
    all_nnzs = []
    for row in range(mat.shape[0]):
        rowdata = []
        colidxs = []
        all_nnzs.append(0)
        for col in range(mat.shape[1]):
            val = mat[row][col]
            if val != 0:
                rowdata.append(val)
                colidxs.append(col)
                all_nnzs[-1] += 1
        all_rows.append(rowdata)
        all_idxs.append(colidxs)
    
    ellwidth = min(int(np.sqrt(np.max(all_nnzs))+1)**2, mat.shape[1])
    #all_rows = np.array(all_rows)#.astype(np.float32).flatten()           
    for row in range(mat.shape[0]):
        #print(row, all_rows)
        all_rows[row] = np.array(all_rows[row])
        all_rows[row].resize(ellwidth)
        all_idxs[row] = np.array(all_idxs[row])
        all_idxs[row].resize(ellwidth)
        print(all_idxs[row])
    all_rows = np.array(all_rows).astype(np.float32).flatten()
    print(all_idxs)
    all_idxs = np.array(all_idxs).astype(np.uint32).flatten()
    all_nnzs = np.array(all_nnzs).astype(np.uint32)
    
    return all_rows, all_idxs, all_nnzs, ellwidth

In [13]:
def to_dense(data, cols, nnzs, ellw, shape):
    out = np.zeros(shape)
    for row in range(shape[0]):
        for icol in range(nnzs[row]):
            out[row,cols[row*ellw+icol]] = data[row*ellw+icol]
    return out

In [14]:
adata, acols, annz, ellwa = to_data(a)
adata, acols, annz, ellwa

[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]
[array([0, 1, 2, 3, 4, 5, 6, 7]), array([0, 1, 2, 3, 4, 5, 6, 7]), array([0, 1, 2, 3, 4, 5, 6, 7]), array([0, 1, 2, 3, 4, 5, 6, 7])]


(array([0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 , 0.6204036 , 0.21273285,
        0.59999454, 0.7183614 , 0.87148714, 0.69184786, 0.04276368,
        0.46898165, 0.02936536, 0.8756672 , 0.41719204, 0.13531114,
        0.09763599, 0.21822612, 0.38778782, 0.0189462 , 0.7515835 ,
        0.12442248, 0.0349297 , 0.37084636, 0.19291036, 0.0798979 ,
        0.6807204 , 0.61051154], dtype=float32),
 array([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5,
        6, 7, 0, 1, 2, 3, 4, 5, 6, 7], dtype=uint32),
 array([8, 8, 8, 8], dtype=uint32),
 8)

In [15]:
adatat, acolst, annzt, ellwat = to_data(a.T)
adatat, acolst, annzt, ellwat

[0 1 2 3]
[0 1 2 3]
[0 1 2 3]
[0 1 2 3]
[0 1 2 3]
[0 1 2 3]
[0 1 2 3]
[0 1 2 3]
[array([0, 1, 2, 3]), array([0, 1, 2, 3]), array([0, 1, 2, 3]), array([0, 1, 2, 3]), array([0, 1, 2, 3]), array([0, 1, 2, 3]), array([0, 1, 2, 3]), array([0, 1, 2, 3])]


(array([0.29516777, 0.6204036 , 0.02936536, 0.7515835 , 0.33338296,
        0.21273285, 0.8756672 , 0.12442248, 0.2910005 , 0.59999454,
        0.41719204, 0.0349297 , 0.5361629 , 0.7183614 , 0.13531114,
        0.37084636, 0.31872442, 0.87148714, 0.09763599, 0.19291036,
        0.00900633, 0.69184786, 0.21822612, 0.0798979 , 0.53607106,
        0.04276368, 0.38778782, 0.6807204 , 0.9474776 , 0.46898165,
        0.0189462 , 0.61051154], dtype=float32),
 array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1,
        2, 3, 0, 1, 2, 3, 0, 1, 2, 3], dtype=uint32),
 array([4, 4, 4, 4, 4, 4, 4, 4], dtype=uint32),
 4)

In [16]:
bdata, bcols, bnnz, ellwb = to_data(b)
bdata, bcols, bnnz, ellwb

[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[array([0, 1, 2]), array([0, 1, 2]), array([0, 1, 2]), array([0, 1, 2]), array([0, 1, 2]), array([0, 1, 2]), array([0, 1, 2]), array([0, 1, 2])]


(array([0.15043414, 0.6196845 , 0.35063207, 0.4796516 , 0.5857214 ,
        0.90562654, 0.08651726, 0.82600874, 0.6354987 , 0.74133885,
        0.2394775 , 0.5392108 , 0.777668  , 0.3423175 , 0.10126331,
        0.2247686 , 0.01690849, 0.9917596 , 0.8016237 , 0.904124  ,
        0.01441843, 0.88494974, 0.20410454, 0.5339545 ], dtype=float32),
 array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0,
        1, 2], dtype=uint32),
 array([3, 3, 3, 3, 3, 3, 3, 3], dtype=uint32),
 3)

In [17]:
bdatat, bcolst, bnnzt, ellwbt = to_data(b.T)
adatat, bcolst, bnnzt, ellwbt

[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]
[array([0, 1, 2, 3, 4, 5, 6, 7]), array([0, 1, 2, 3, 4, 5, 6, 7]), array([0, 1, 2, 3, 4, 5, 6, 7])]


(array([0.29516777, 0.6204036 , 0.02936536, 0.7515835 , 0.33338296,
        0.21273285, 0.8756672 , 0.12442248, 0.2910005 , 0.59999454,
        0.41719204, 0.0349297 , 0.5361629 , 0.7183614 , 0.13531114,
        0.37084636, 0.31872442, 0.87148714, 0.09763599, 0.19291036,
        0.00900633, 0.69184786, 0.21822612, 0.0798979 , 0.53607106,
        0.04276368, 0.38778782, 0.6807204 , 0.9474776 , 0.46898165,
        0.0189462 , 0.61051154], dtype=float32),
 array([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5,
        6, 7], dtype=uint32),
 array([8, 8, 8], dtype=uint32),
 8)

In [18]:
adense = to_dense(adata, acols, annz, ellwa, a.shape)

In [19]:
adenset = to_dense(adatat, acolst, annzt, ellwat, a.T.shape)

In [20]:
bdense = to_dense(bdata, bcols, bnnz, ellwb, b.shape)

In [21]:
bdenset = to_dense(bdatat, bcolst, bnnzt, ellwbt, b.T.shape)

In [22]:
adense

array([[0.29516777, 0.33338296, 0.29100049, 0.53616291, 0.31872442,
        0.00900633, 0.53607106, 0.94747758],
       [0.62040359, 0.21273285, 0.59999454, 0.71836138, 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.87566721, 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.75158352, 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.68072039, 0.61051154]])

In [23]:
adenset.T == adense

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

In [24]:
bdenset.T == bdense

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [25]:
a

array([[0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 ],
       [0.6204036 , 0.21273285, 0.59999454, 0.7183614 , 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.8756672 , 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.7515835 , 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.6807204 , 0.61051154]], dtype=float32)

In [26]:
a == adense

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

In [27]:
a.shape

(4, 8)

In [28]:
adata.shape, acols.shape, annz.shape, ellwa

((32,), (32,), (4,), 8)

In [29]:
#acols = acols.astype(np.uint32)
#annz = annz.astype(np.uint32)

In [30]:
adata, acols, annz, b

(array([0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 , 0.6204036 , 0.21273285,
        0.59999454, 0.7183614 , 0.87148714, 0.69184786, 0.04276368,
        0.46898165, 0.02936536, 0.8756672 , 0.41719204, 0.13531114,
        0.09763599, 0.21822612, 0.38778782, 0.0189462 , 0.7515835 ,
        0.12442248, 0.0349297 , 0.37084636, 0.19291036, 0.0798979 ,
        0.6807204 , 0.61051154], dtype=float32),
 array([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5,
        6, 7, 0, 1, 2, 3, 4, 5, 6, 7], dtype=uint32),
 array([8, 8, 8, 8], dtype=uint32),
 array([[0.15043414, 0.6196845 , 0.35063207],
        [0.4796516 , 0.5857214 , 0.90562654],
        [0.08651726, 0.82600874, 0.6354987 ],
        [0.74133885, 0.2394775 , 0.5392108 ],
        [0.777668  , 0.3423175 , 0.10126331],
        [0.2247686 , 0.01690849, 0.9917596 ],
        [0.8016237 , 0.904124  , 0.01441843],
        [0.88494974, 0.20410454, 0.5339545 ]], dtype=floa

## MatMul (Sparse-Dense)

adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // SPARSE x DENSE
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      

      uint nnz    = rowNnz[gid];
      float sum = 0;
      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[col*ncols+gid2];
          //if (gid==0 && gid2==2)
          //  printf("aval, xval: %.2f,%.2f: (%i,%i) \\n", aval, xval, col, index);
          sum  += aval * xval;
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [31]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // SPARSE x DENSE
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      

      uint nnz    = rowNnz[gid];
      float sum = 0;
      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[col*ncols+gid2];
          if (gid==0 && gid2==1)
            printf("aval, xval: %.2f,%.2f: (%i,%i) \\n", aval, xval, col, index);
          sum  += aval * xval;
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [32]:
a.shape, b.shape

((4, 8), (8, 3))

In [33]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [34]:
rows = a.shape[0]

In [35]:
mult = mult.astype(np.float32)

In [36]:
outshape = (a.shape[0], b.shape[1])
outshape

(4, 3)

In [37]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul2  # Use this Kernel object for repeated calls
knl(queue, [outshape[0]], None, adata_buf, acols_buf, annzs_buf, np.uint32(ellwa), np.uint32(outshape[1]), b_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

aval, xval: 0.30,0.62: (0,0) 
aval, xval: 0.33,0.59: (1,1) 
aval, xval: 0.29,0.83: (2,2) 
aval, xval: 0.54,0.24: (3,3) 
aval, xval: 0.32,0.34: (4,4) 
aval, xval: 0.01,0.02: (5,5) 
aval, xval: 0.54,0.90: (6,6) 
aval, xval: 0.95,0.20: (7,7) 


In [38]:
(res_np-mult).sum()

19.769743

In [39]:
a

array([[0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 ],
       [0.6204036 , 0.21273285, 0.59999454, 0.7183614 , 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.8756672 , 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.7515835 , 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.6807204 , 0.61051154]], dtype=float32)

In [40]:
b

array([[0.15043414, 0.6196845 , 0.35063207],
       [0.4796516 , 0.5857214 , 0.90562654],
       [0.08651726, 0.82600874, 0.6354987 ],
       [0.74133885, 0.2394775 , 0.5392108 ],
       [0.777668  , 0.3423175 , 0.10126331],
       [0.2247686 , 0.01690849, 0.9917596 ],
       [0.8016237 , 0.904124  , 0.01441843],
       [0.88494974, 0.20410454, 0.5339545 ]], dtype=float32)

In [41]:
res_buf

In [42]:
res_np

array([[2.1450493, 3.679314 , 5.1136117],
       [2.0623655, 3.6834624, 5.887724 ],
       [1.0134436, 2.3131337, 3.6965663],
       [1.7046205, 3.168355 , 4.201301 ]], dtype=float32)

In [43]:
mult

array([[2.1450493, 1.5342648, 1.4342973],
       [2.0623655, 1.6210971, 2.2042618],
       [1.0134436, 1.2996901, 1.3834326],
       [1.7046205, 1.4637345, 1.0329463]], dtype=float32)

In [44]:
res_np==mult

array([[ True, False, False],
       [ True, False, False],
       [ True, False, False],
       [ True, False, False]])

In [45]:
res_np.shape

(4, 3)

In [46]:
mult.shape

(4, 3)

## MatMul (dense * sparse)

In [47]:
bdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdata)
bcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcols)
bnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnz)
bdatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdatat)
bcolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcolst)
bnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnzt)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // DENSE x SPARSE
    __kernel void matmul(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint   mwidth,
                            uint   ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      uint nnz    = rowNnz[gid];
      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        float sum = 0;
        for (uint i = 0; i < mwidth; i++) {
          uint index   = (gid2 * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[gid*mwidth+col];
          sum  += aval * xval;
          if (gid==0 && gid2==0)
            printf("aval, xval: %.2f,%.2f - %.2f: (%i,%i) \\n", aval, xval, sum, col, index);
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [48]:
a.shape, b.shape

((4, 8), (8, 3))

In [49]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [50]:
rows = a.shape[0]

In [51]:
mult = mult.astype(np.float32)

In [52]:
outshape = np.array([a.shape[0], b.shape[1]])
outshape

array([4, 3])

In [53]:
b.T

array([[0.15043414, 0.4796516 , 0.08651726, 0.74133885, 0.777668  ,
        0.2247686 , 0.8016237 , 0.88494974],
       [0.6196845 , 0.5857214 , 0.82600874, 0.2394775 , 0.3423175 ,
        0.01690849, 0.904124  , 0.20410454],
       [0.35063207, 0.90562654, 0.6354987 , 0.5392108 , 0.10126331,
        0.9917596 , 0.01441843, 0.5339545 ]], dtype=float32)

In [54]:
a.T

array([[0.29516777, 0.6204036 , 0.02936536, 0.7515835 ],
       [0.33338296, 0.21273285, 0.8756672 , 0.12442248],
       [0.2910005 , 0.59999454, 0.41719204, 0.0349297 ],
       [0.5361629 , 0.7183614 , 0.13531114, 0.37084636],
       [0.31872442, 0.87148714, 0.09763599, 0.19291036],
       [0.00900633, 0.69184786, 0.21822612, 0.0798979 ],
       [0.53607106, 0.04276368, 0.38778782, 0.6807204 ],
       [0.9474776 , 0.46898165, 0.0189462 , 0.61051154]], dtype=float32)

In [55]:
outshape.T

array([4, 3])

In [56]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul  # Use this Kernel object for repeated calls
knl(queue, [outshape.T[0]], None, bdatat_buf, bcolst_buf, bnnzst_buf, np.uint32(ellwbt), np.uint32(b.shape[0]), np.uint32(outshape.T[1]), a_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

aval, xval: 0.15,0.30 - 0.04: (0,0) 
aval, xval: 0.48,0.33 - 0.20: (1,1) 
aval, xval: 0.09,0.29 - 0.23: (2,2) 
aval, xval: 0.74,0.54 - 0.63: (3,3) 
aval, xval: 0.78,0.32 - 0.87: (4,4) 
aval, xval: 0.22,0.01 - 0.88: (5,5) 
aval, xval: 0.80,0.54 - 1.31: (6,6) 
aval, xval: 0.88,0.95 - 2.15: (7,7) 


In [57]:
(res_np-mult).sum()

0.0

In [58]:
a

array([[0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 ],
       [0.6204036 , 0.21273285, 0.59999454, 0.7183614 , 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.8756672 , 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.7515835 , 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.6807204 , 0.61051154]], dtype=float32)

In [59]:
b

array([[0.15043414, 0.6196845 , 0.35063207],
       [0.4796516 , 0.5857214 , 0.90562654],
       [0.08651726, 0.82600874, 0.6354987 ],
       [0.74133885, 0.2394775 , 0.5392108 ],
       [0.777668  , 0.3423175 , 0.10126331],
       [0.2247686 , 0.01690849, 0.9917596 ],
       [0.8016237 , 0.904124  , 0.01441843],
       [0.88494974, 0.20410454, 0.5339545 ]], dtype=float32)

In [60]:
res_buf

In [61]:
res_np

array([[2.1450493, 1.5342648, 1.4342973],
       [2.0623655, 1.6210971, 2.2042618],
       [1.0134436, 1.2996901, 1.3834326],
       [1.7046205, 1.4637345, 1.0329463]], dtype=float32)

In [62]:
mult

array([[2.1450493, 1.5342648, 1.4342973],
       [2.0623655, 1.6210971, 2.2042618],
       [1.0134436, 1.2996901, 1.3834326],
       [1.7046205, 1.4637345, 1.0329463]], dtype=float32)

In [63]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [64]:
res_np.shape

(4, 3)

In [65]:
mult.shape

(4, 3)

# Matmult Dense Dense

In [66]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      uint osize = get_global_size(1);
      int gidx = get_global_id(0); // row
      int gidy = get_global_id(1); // col

      float ret = 0.0;
      for (int i = 0; i < msize; i++) {
        uint xidx = gidx*msize+i; 
        float xval = x[xidx];
        uint yidx = osize*i+gidy;
        float yval = y[yidx];
        ret += xval*yval;
        if (gidx==0 && gidy==0)
          printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, res, xidx, yidx);
      }

      //if (gidx==0&&gidy==0)
      //  printf("\\nsum:%.2f", ret);
      res[gidx * osize + gidy] = ret;
    }""").build()

In [67]:
a.shape, b.shape

((4, 8), (8, 3))

In [68]:
rows = a.shape[0]

In [69]:
mult = mult.astype(np.float32)

In [70]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows,b.shape[1]], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.30 x 0.15 - 0.00  -- 0/0
mult: 0.33 x 0.48 - 0.00  -- 1/3
mult: 0.29 x 0.09 - 0.00  -- 2/6
mult: 0.54 x 0.74 - 0.00  -- 3/9
mult: 0.32 x 0.78 - 0.00  -- 4/12
mult: 0.01 x 0.22 - 0.00  -- 5/15
mult: 0.54 x 0.80 - 0.00  -- 6/18
mult: 0.95 x 0.88 - 0.00  -- 7/21

In [71]:
(res_np-mult).sum()

0.0

In [72]:
a

array([[0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 ],
       [0.6204036 , 0.21273285, 0.59999454, 0.7183614 , 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.8756672 , 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.7515835 , 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.6807204 , 0.61051154]], dtype=float32)

In [73]:
b

array([[0.15043414, 0.6196845 , 0.35063207],
       [0.4796516 , 0.5857214 , 0.90562654],
       [0.08651726, 0.82600874, 0.6354987 ],
       [0.74133885, 0.2394775 , 0.5392108 ],
       [0.777668  , 0.3423175 , 0.10126331],
       [0.2247686 , 0.01690849, 0.9917596 ],
       [0.8016237 , 0.904124  , 0.01441843],
       [0.88494974, 0.20410454, 0.5339545 ]], dtype=float32)

In [74]:
res_np

array([[2.1450493, 1.5342648, 1.4342973],
       [2.0623655, 1.6210971, 2.2042618],
       [1.0134436, 1.2996901, 1.3834326],
       [1.7046205, 1.4637345, 1.0329463]], dtype=float32)

In [75]:
a.dot(b)

array([[2.1450493, 1.5342648, 1.4342973],
       [2.0623655, 1.6210971, 2.2042618],
       [1.0134436, 1.2996901, 1.3834326],
       [1.7046205, 1.4637345, 1.0329463]], dtype=float32)

In [76]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [77]:
res_np.shape

(4, 3)

In [78]:
mult.shape

(4, 3)

# Matmult Dense Transposed

In [79]:
b

array([[0.15043414, 0.6196845 , 0.35063207],
       [0.4796516 , 0.5857214 , 0.90562654],
       [0.08651726, 0.82600874, 0.6354987 ],
       [0.74133885, 0.2394775 , 0.5392108 ],
       [0.777668  , 0.3423175 , 0.10126331],
       [0.2247686 , 0.01690849, 0.9917596 ],
       [0.8016237 , 0.904124  , 0.01441843],
       [0.88494974, 0.20410454, 0.5339545 ]], dtype=float32)

In [80]:
c=np.zeros(b.T.shape)
bt = b.T
for row in range(bt.shape[0]):
    for col in range(bt.shape[1]):
        c[row][col] = bt[row][col]

In [81]:
bt

array([[0.15043414, 0.4796516 , 0.08651726, 0.74133885, 0.777668  ,
        0.2247686 , 0.8016237 , 0.88494974],
       [0.6196845 , 0.5857214 , 0.82600874, 0.2394775 , 0.3423175 ,
        0.01690849, 0.904124  , 0.20410454],
       [0.35063207, 0.90562654, 0.6354987 , 0.5392108 , 0.10126331,
        0.9917596 , 0.01441843, 0.5339545 ]], dtype=float32)

In [82]:
c

array([[0.15043414, 0.4796516 , 0.08651726, 0.74133885, 0.777668  ,
        0.22476859, 0.8016237 , 0.88494974],
       [0.61968452, 0.58572137, 0.82600874, 0.2394775 , 0.34231749,
        0.01690849, 0.90412402, 0.20410454],
       [0.35063207, 0.90562654, 0.6354987 , 0.5392108 , 0.10126331,
        0.9917596 , 0.01441843, 0.5339545 ]])

In [83]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      uint osize = get_global_size(1);
      int gidx = get_global_id(0); // row
      int gidy = get_global_id(1); // col

      float ret = 0.0;
      for (int i = 0; i < msize; i++) {
        uint xidx = gidx*msize+i;
        float xval = x[xidx];
        uint yidx = msize*gidy+i;
        float yval = y[yidx];
        ret += xval*yval;
        if (gidx==0 && gidy==0)
          printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, res, xidx, yidx);
      }

      //if (gidx==0&&gidy==0)
      //  printf("\\nsum:%.2f", ret);
      res[gidx * osize + gidy] = ret;
    }""").build()

In [84]:
a.shape, b.T.shape

((4, 8), (3, 8))

In [85]:
rows = a.shape[0]

In [86]:
mult = mult.astype(np.float32)

In [87]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows,b.shape[1]], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.30 x 0.15 - 0.00  -- 0/0
mult: 0.33 x 0.48 - 0.00  -- 1/1
mult: 0.29 x 0.09 - 0.00  -- 2/2
mult: 0.54 x 0.74 - 0.00  -- 3/3
mult: 0.32 x 0.78 - 0.00  -- 4/4
mult: 0.01 x 0.22 - 0.00  -- 5/5
mult: 0.54 x 0.80 - 0.00  -- 6/6
mult: 0.95 x 0.88 - 0.00  -- 7/7

In [88]:
(res_np-mult).sum()

0.0

In [89]:
a

array([[0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 ],
       [0.6204036 , 0.21273285, 0.59999454, 0.7183614 , 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.8756672 , 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.7515835 , 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.6807204 , 0.61051154]], dtype=float32)

In [90]:
b

array([[0.15043414, 0.6196845 , 0.35063207],
       [0.4796516 , 0.5857214 , 0.90562654],
       [0.08651726, 0.82600874, 0.6354987 ],
       [0.74133885, 0.2394775 , 0.5392108 ],
       [0.777668  , 0.3423175 , 0.10126331],
       [0.2247686 , 0.01690849, 0.9917596 ],
       [0.8016237 , 0.904124  , 0.01441843],
       [0.88494974, 0.20410454, 0.5339545 ]], dtype=float32)

In [91]:
res_np

array([[2.1450493, 1.5342648, 1.4342973],
       [2.0623655, 1.6210971, 2.2042618],
       [1.0134436, 1.2996901, 1.3834326],
       [1.7046205, 1.4637345, 1.0329463]], dtype=float32)

In [92]:
a.dot(b)

array([[2.1450493, 1.5342648, 1.4342973],
       [2.0623655, 1.6210971, 2.2042618],
       [1.0134436, 1.2996901, 1.3834326],
       [1.7046205, 1.4637345, 1.0329463]], dtype=float32)

In [93]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [94]:
res_np.shape

(4, 3)

In [95]:
mult.shape

(4, 3)

# Matmult Transposed Dense

In [96]:
b

array([[0.15043414, 0.6196845 , 0.35063207],
       [0.4796516 , 0.5857214 , 0.90562654],
       [0.08651726, 0.82600874, 0.6354987 ],
       [0.74133885, 0.2394775 , 0.5392108 ],
       [0.777668  , 0.3423175 , 0.10126331],
       [0.2247686 , 0.01690849, 0.9917596 ],
       [0.8016237 , 0.904124  , 0.01441843],
       [0.88494974, 0.20410454, 0.5339545 ]], dtype=float32)

In [97]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [98]:
at

array([[0.29516777, 0.6204036 , 0.02936536, 0.7515835 ],
       [0.33338296, 0.21273285, 0.8756672 , 0.12442248],
       [0.2910005 , 0.59999454, 0.41719204, 0.0349297 ],
       [0.5361629 , 0.7183614 , 0.13531114, 0.37084636],
       [0.31872442, 0.87148714, 0.09763599, 0.19291036],
       [0.00900633, 0.69184786, 0.21822612, 0.0798979 ],
       [0.53607106, 0.04276368, 0.38778782, 0.6807204 ],
       [0.9474776 , 0.46898165, 0.0189462 , 0.61051154]], dtype=float32)

In [99]:
c

array([[0.29516777, 0.62040359, 0.02936536, 0.75158352],
       [0.33338296, 0.21273285, 0.87566721, 0.12442248],
       [0.29100049, 0.59999454, 0.41719204, 0.0349297 ],
       [0.53616291, 0.71836138, 0.13531114, 0.37084636],
       [0.31872442, 0.87148714, 0.09763599, 0.19291036],
       [0.00900633, 0.69184786, 0.21822612, 0.0798979 ],
       [0.53607106, 0.04276368, 0.38778782, 0.68072039],
       [0.94747758, 0.46898165, 0.0189462 , 0.61051154]])

In [100]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // multilplies x TRANSPOSED by y (dense-dense)
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize,
                          uint isize
                          ) { // LOCAL SHARED BUFFER
      uint osize = get_global_size(0);
      int gidy = get_global_id(0); // row
      
      for (uint gidx = 0; gidx < isize; gidx++) {
        float ret = 0.0;
        for (uint i = 0; i < msize; i++) {
          uint xidx = i*isize+gidx;
          float xval = x[xidx];
          uint yidx = osize*i+gidy;
          float yval = y[yidx];
          ret += xval*yval;
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, ret, xidx, yidx);
        }
        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        res[gidx * osize + gidy] = ret;
      }
    }""").build()

In [101]:
a.shape, b.T.shape

((4, 8), (3, 8))

In [102]:
rows = a.shape[0]

In [103]:
mult = mult.astype(np.float32)

In [104]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [b.shape[1]], None, a_buf, b_buf, res_buf, np.uint32(a.shape[1]), np.uint32(rows))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.30 x 0.15 - 0.04  -- 0/0
mult: 0.33 x 0.48 - 0.20  -- 4/3
mult: 0.29 x 0.09 - 0.23  -- 8/6
mult: 0.54 x 0.74 - 0.63  -- 12/9
mult: 0.32 x 0.78 - 0.87  -- 16/12
mult: 0.01 x 0.22 - 0.88  -- 20/15
mult: 0.54 x 0.80 - 1.31  -- 24/18
mult: 0.95 x 0.88 - 2.15  -- 28/21

In [105]:
(res_np-mult).sum()

0.0

In [106]:
a

array([[0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 ],
       [0.6204036 , 0.21273285, 0.59999454, 0.7183614 , 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.8756672 , 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.7515835 , 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.6807204 , 0.61051154]], dtype=float32)

In [107]:
b

array([[0.15043414, 0.6196845 , 0.35063207],
       [0.4796516 , 0.5857214 , 0.90562654],
       [0.08651726, 0.82600874, 0.6354987 ],
       [0.74133885, 0.2394775 , 0.5392108 ],
       [0.777668  , 0.3423175 , 0.10126331],
       [0.2247686 , 0.01690849, 0.9917596 ],
       [0.8016237 , 0.904124  , 0.01441843],
       [0.88494974, 0.20410454, 0.5339545 ]], dtype=float32)

In [108]:
res_np

array([[2.1450493, 1.5342648, 1.4342973],
       [2.0623655, 1.6210971, 2.2042618],
       [1.0134436, 1.2996901, 1.3834326],
       [1.7046205, 1.4637345, 1.0329463]], dtype=float32)

In [109]:
a.dot(b)

array([[2.1450493, 1.5342648, 1.4342973],
       [2.0623655, 1.6210971, 2.2042618],
       [1.0134436, 1.2996901, 1.3834326],
       [1.7046205, 1.4637345, 1.0329463]], dtype=float32)

In [110]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [111]:
res_np.shape

(4, 3)

In [112]:
mult.shape

(4, 3)

# Matmult Dense Transposed2

In [113]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize,
                          uint osize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      // osize = get_global_size(1);
      int gidx = get_global_id(0); // col
      // int gidy = get_global_id(1); // row

      for (uint gidy = 0; gidy < osize; gidy++) {
        float ret = 0.0;
        for (uint i = 0; i < msize; i++) {
          ret += x[gidx*msize+i]*y[i*osize+gidy];
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f", x[gidx*msize+i],y[i*msize+gidy], ret);
        }

        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        res[gidx * osize + gidy] = ret;
      }
    }""").build()

In [114]:
a.shape, b.shape

((4, 8), (8, 3))

In [115]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [116]:
rows = a.shape[0]

In [117]:
mult = mult.astype(np.float32)

In [118]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]), np.uint32(b.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.30 x 0.15 - 0.04
mult: 0.33 x 0.64 - 0.20
mult: 0.29 x 0.02 - 0.23
mult: 0.54 x 0.00 - 0.63
mult: 0.32 x 0.00 - 0.87
mult: 0.01 x 0.00 - 0.88
mult: 0.54 x 0.00 - 1.31
mult: 0.95 x 0.00 - 2.15

In [119]:
(res_np-mult).sum()

0.0

In [120]:
a

array([[0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 ],
       [0.6204036 , 0.21273285, 0.59999454, 0.7183614 , 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.8756672 , 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.7515835 , 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.6807204 , 0.61051154]], dtype=float32)

In [121]:
b.T

array([[0.15043414, 0.4796516 , 0.08651726, 0.74133885, 0.777668  ,
        0.2247686 , 0.8016237 , 0.88494974],
       [0.6196845 , 0.5857214 , 0.82600874, 0.2394775 , 0.3423175 ,
        0.01690849, 0.904124  , 0.20410454],
       [0.35063207, 0.90562654, 0.6354987 , 0.5392108 , 0.10126331,
        0.9917596 , 0.01441843, 0.5339545 ]], dtype=float32)

In [122]:
a[0]

array([0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
       0.00900633, 0.53607106, 0.9474776 ], dtype=float32)

In [123]:
b.T[0]

array([0.15043414, 0.4796516 , 0.08651726, 0.74133885, 0.777668  ,
       0.2247686 , 0.8016237 , 0.88494974], dtype=float32)

In [124]:
res_buf

In [125]:
res_np

array([[2.1450493, 1.5342648, 1.4342973],
       [2.0623655, 1.6210971, 2.2042618],
       [1.0134436, 1.2996901, 1.3834326],
       [1.7046205, 1.4637345, 1.0329463]], dtype=float32)

In [126]:
a.dot(b)

array([[2.1450493, 1.5342648, 1.4342973],
       [2.0623655, 1.6210971, 2.2042618],
       [1.0134436, 1.2996901, 1.3834326],
       [1.7046205, 1.4637345, 1.0329463]], dtype=float32)

In [127]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [128]:
res_np.shape

(4, 3)

In [129]:
mult.shape

(4, 3)

## Weight update kernel

In [130]:
bs = 4

In [131]:
dim = 8
topk = 2

x = np.random.rand(bs,dim).astype(np.float32)
y = np.random.rand(bs,dim).astype(np.float32)
x.shape,y.shape, topk

((4, 8), (4, 8), 2)

x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint bs = get_global_size(1);
      uint gid2 = get_global_id(1);

      uint idx = n*gid2+gid;

      float valx = x[idx];
      float valy = y[idx];
      uint posx = 0;
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval = x[idx2];
        float tempval2 = y[idx2];
        bool larger = tempval > valx;
        bool larger2 = tempval2 > valy;

        barrier(CLK_GLOBAL_MEM_FENCE);
        posx += (larger)?1:0;
        posy += (larger2)?1:0;
        barrier(CLK_GLOBAL_MEM_FENCE);
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      //printf("posx:%i", posx);
      if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
      }
      if (posy < topk) {
        youtidx[posy+topk*gid2] = gid;
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      if (gid < topk) {
        for (uint j=0; j<topk; j++) {
          float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
          //printf("\\nJ:%i  gid:%i", j, gid);
          //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
          barrier(CLK_GLOBAL_MEM_FENCE);
          xout[gid2*topk*topk+j*topk+gid] = res;
          barrier(CLK_GLOBAL_MEM_FENCE);
          
        }
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
    }""").build()

In [132]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
#x_cp_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
#x_idx_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
#y_idx_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             uint bs,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      //uint bs = get_global_size(1);
      //uint gid2 = get_global_id(1);

      for (uint gid2=0; gid2<bs; gid2++){
        uint idx = n*gid2+gid;

        float valx = x[idx];
        float valy = y[idx];
        uint posx = 0;
        uint posy = 0;
        for (uint i = 0; i < n; i++) {
          uint idx2 = n*gid2+i;
          float tempval = x[idx2];
          float tempval2 = y[idx2];
          bool larger = tempval > valx;
          bool larger2 = tempval2 > valy;

          posx += (larger)?1:0;
          posy += (larger2)?1:0;
        }
        //printf("posx:%i", posx);
        if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
        }
        if (posy < topk) {
          youtidx[posy+topk*gid2] = gid;
        }
      }
      for (uint gid2=0; gid2<bs; gid2++){
        if (gid < topk) {
          for (uint j=0; j<topk; j++) {
            float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
            //printf("\\nJ:%i  gid:%i", j, gid);
            //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
            //barrier(CLK_GLOBAL_MEM_FENCE);
            xout[gid2*topk*topk+j*topk+gid] = res;
          }
        }
      }
    }""").build()

In [133]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
evt = knl(queue, [dim], None, x_buf, y_buf, x_cp_buf, np.uint32(topk), np.uint32(bs), x_idx_buf, y_idx_buf)

#evt.wait()
resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

knl(queue, [dim], None, y_buf, x_buf, x_cp_buft, np.uint32(topk), np.uint32(bs), x_idx_buft, y_idx_buft)

#evt.wait()
resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [134]:
x

array([[0.5208921 , 0.3572506 , 0.30112275, 0.11082315, 0.25933608,
        0.2537252 , 0.8020481 , 0.39789936],
       [0.61094195, 0.0611377 , 0.8704709 , 0.8843079 , 0.417089  ,
        0.6639791 , 0.15737332, 0.02135541],
       [0.1559249 , 0.14806202, 0.8460671 , 0.9979287 , 0.7261287 ,
        0.12866202, 0.8465165 , 0.66926867],
       [0.84276325, 0.06656174, 0.16729096, 0.13972558, 0.38513017,
        0.5822583 , 0.76684874, 0.45251068]], dtype=float32)

In [135]:
y

array([[0.78604454, 0.00869259, 0.31350958, 0.74469817, 0.41800404,
        0.6869445 , 0.26334533, 0.36119112],
       [0.51565766, 0.10113762, 0.7259253 , 0.23368242, 0.5307467 ,
        0.25279608, 0.17987116, 0.7080778 ],
       [0.8590547 , 0.12702702, 0.09493987, 0.75800353, 0.04770023,
        0.8330679 , 0.67851025, 0.03306201],
       [0.1939566 , 0.02938453, 0.57044244, 0.15289223, 0.10998691,
        0.32446983, 0.6489413 , 0.6973374 ]], dtype=float32)

In [136]:
x.shape, y.shape

((4, 8), (4, 8))

In [137]:
resx

array([0.63044554, 0.4094444 , 0.5972837 , 0.3879074 , 0.6419415 ,
       0.63189685, 0.6261588 , 0.6163611 , 0.8572753 , 0.72720397,
       0.83134234, 0.70520574, 0.5876903 , 0.5347523 , 0.54690385,
       0.4976398 ], dtype=float32)

In [138]:
resx.reshape(bs,topk,topk)

array([[[0.63044554, 0.4094444 ],
        [0.5972837 , 0.3879074 ]],

       [[0.6419415 , 0.63189685],
        [0.6261588 , 0.6163611 ]],

       [[0.8572753 , 0.72720397],
        [0.83134234, 0.70520574]],

       [[0.5876903 , 0.5347523 ],
        [0.54690385, 0.4976398 ]]], dtype=float32)

In [139]:
resxidx

array([6, 0, 3, 2, 3, 6, 0, 6], dtype=uint32)

In [140]:
resyidx

array([0, 3, 2, 7, 0, 5, 7, 6], dtype=uint32)

In [141]:
idx = 1
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

(8, 8)

In [142]:
xy0[3][7]

0.6261588

### update vals add dense

In [143]:
matadd = np.random.randn(*a.shape).astype(np.float32)
matadd

array([[ 0.13292159, -0.20273261,  1.0448247 ,  0.4868517 ,  0.7249878 ,
         1.6218207 ,  0.41703546, -1.1581254 ],
       [ 1.8033321 ,  1.2297767 ,  0.3011195 ,  0.5693663 ,  1.025062  ,
        -2.1702912 , -1.7832675 ,  0.7361344 ],
       [ 0.5765973 , -0.77569616,  0.5184834 , -1.2598531 ,  0.07274681,
        -1.5193717 , -0.79238296, -0.6951347 ],
       [ 0.05345648,  0.21328321, -1.9485346 ,  0.6406955 ,  1.1379646 ,
        -0.21245147, -0.12579903,  1.1361357 ]], dtype=float32)

In [144]:
a

array([[0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 ],
       [0.6204036 , 0.21273285, 0.59999454, 0.7183614 , 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.8756672 , 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.7515835 , 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.6807204 , 0.61051154]], dtype=float32)

In [145]:
a_added = a + matadd

In [146]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
add_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=matadd)

prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddense(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            uint   awidth,
                            __global  float* vector_x    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      uint nnz    = rowNnz[gid];
      uint baseidxs = gid*ellwidth;
      uint baseidxd = gid*awidth;
      
      for (uint i=0; i<awidth; i++) {
        float addval = vector_x[baseidxd+i];
        //if (gid==1)
        //  printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[baseidxs+i]);
        if (addval == 0) {
          continue;
        }
        if (i == colIdx[baseidxs+i]) {
          matData[baseidxs+i] += addval;
        } else {
          if (i > colIdx[baseidxs+i])
            break;
          for (uint j=nnz; j>i; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          //if (gid==1)
          //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] = addval;
          colIdx[baseidxs+i] = i;
          if (nnz >= ellwidth)
            break;
        }
      }
    }""").build()

In [147]:
a.shape, b.shape

((4, 8), (8, 3))

In [148]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [149]:
rows = a.shape[0]

In [150]:
mult = mult.astype(np.float32)

In [151]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddense  # Use this Kernel object for repeated calls
knl(queue, [rows], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa),np.uint32(a.shape[1]), add_buf)

In [152]:
matadd[0][0]

0.13292159

In [153]:
data_res = np.empty_like(adata)
cols_res = np.empty_like(acols)
nnzs_res = np.empty_like(annz)
cl.enqueue_copy(queue, data_res, adata_buf)
cl.enqueue_copy(queue, cols_res, acols_buf)
cl.enqueue_copy(queue, nnzs_res, annzs_buf)

In [154]:
adenseadd = to_dense(data_res, cols_res, nnzs_res, ellwa, a.shape)
adenseadd

array([[ 0.42808938,  0.13065036,  1.3358252 ,  1.02301455,  1.04371226,
         1.63082707,  0.95310652, -0.21064782],
       [ 2.42373562,  1.44250965,  0.90111405,  1.28772759,  1.89654911,
        -1.47844338, -1.74050379,  1.20511603],
       [ 0.60596263,  0.09997106,  0.93567544, -1.124542  ,  0.1703828 ,
        -1.30114567, -0.40459514, -0.67618853],
       [ 0.80504   ,  0.3377057 , -1.91360486,  1.01154184,  1.33087492,
        -0.13255358,  0.55492139,  1.74664724]])

In [155]:
a

array([[0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 ],
       [0.6204036 , 0.21273285, 0.59999454, 0.7183614 , 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.8756672 , 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.7515835 , 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.6807204 , 0.61051154]], dtype=float32)

In [156]:
matadd

array([[ 0.13292159, -0.20273261,  1.0448247 ,  0.4868517 ,  0.7249878 ,
         1.6218207 ,  0.41703546, -1.1581254 ],
       [ 1.8033321 ,  1.2297767 ,  0.3011195 ,  0.5693663 ,  1.025062  ,
        -2.1702912 , -1.7832675 ,  0.7361344 ],
       [ 0.5765973 , -0.77569616,  0.5184834 , -1.2598531 ,  0.07274681,
        -1.5193717 , -0.79238296, -0.6951347 ],
       [ 0.05345648,  0.21328321, -1.9485346 ,  0.6406955 ,  1.1379646 ,
        -0.21245147, -0.12579903,  1.1361357 ]], dtype=float32)

In [157]:
a_added

array([[ 0.42808938,  0.13065036,  1.3358252 ,  1.0230145 ,  1.0437123 ,
         1.6308271 ,  0.9531065 , -0.21064782],
       [ 2.4237356 ,  1.4425097 ,  0.90111405,  1.2877276 ,  1.8965491 ,
        -1.4784434 , -1.7405038 ,  1.205116  ],
       [ 0.60596263,  0.09997106,  0.93567544, -1.124542  ,  0.1703828 ,
        -1.3011457 , -0.40459514, -0.6761885 ],
       [ 0.80504   ,  0.3377057 , -1.9136049 ,  1.0115418 ,  1.3308749 ,
        -0.13255358,  0.5549214 ,  1.7466472 ]], dtype=float32)

In [158]:
adenseadd == a_added

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

### update vals

In [159]:
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
add_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=matadd)

prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddenset(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            uint   aheight,
                            __global  float* vector_x    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint ncols = get_global_size(0);

      uint nnz    = rowNnz[gid];
      uint baseidxs = gid*ellwidth;
      
      for (uint i=0; i<aheight; i++) {
        if (nnz > ellwidth)
            break;
        uint baseidxd = i*ncols+gid;
        float addval = vector_x[baseidxd];
        //if (gid==1)
        //  printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[baseidxs+i]);
        if (addval == 0) {
          continue;
        }
        if (i == colIdx[baseidxs+i]) {
          printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] += addval;
        } else {
          if (i > colIdx[baseidxs+i]) {
            rowNnz[gid] += 1;
            //if (gid==1)
            //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
            matData[baseidxs+i] = addval;
            colIdx[baseidxs+i] = i;
            if (nnz >= ellwidth)
              break;
          }
          for (uint j=nnz; j>i; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          //if (gid==1)
          //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] = addval;
          colIdx[baseidxs+i] = i;
        }
      }
    }""").build()

In [160]:
a.shape, b.shape

((4, 8), (8, 3))

In [161]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [162]:
cols = a.shape[1]

In [163]:
mult = mult.astype(np.float32)

In [164]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddenset  # Use this Kernel object for repeated calls
knl(queue, [cols], None, adatat_buf, acolst_buf, annzst_buf, np.float32(1), np.uint32(ellwat),np.uint32(a.T.shape[1]), add_buf)


ADD VAL:0.13 idx:0/0  col:0
ADD VAL:-0.20 idx:4/1  col:0
ADD VAL:1.04 idx:8/2  col:0
ADD VAL:0.49 idx:12/3  col:0
ADD VAL:0.72 idx:16/4  col:0
ADD VAL:1.62 idx:20/5  col:0
ADD VAL:0.42 idx:24/6  col:0
ADD VAL:-1.16 idx:28/7  col:0
ADD VAL:1.80 idx:1/9  col:1
ADD VAL:1.23 idx:5/10  col:1
ADD VAL:0.30 idx:9/11  col:1
ADD VAL:0.57 idx:13/12  col:1
ADD VAL:1.03 idx:17/13  col:1
ADD VAL:-2.17 idx:21/14  col:1
ADD VAL:-1.78 idx:25/15  col:1
ADD VAL:0.74 idx:29/16  col:1
ADD VAL:0.58 idx:2/18  col:2
ADD VAL:-0.78 idx:6/19  col:2
ADD VAL:0.52 idx:10/20  col:2
ADD VAL:-1.26 idx:14/21  col:2
ADD VAL:0.07 idx:18/22  col:2
ADD VAL:-1.52 idx:22/23  col:2
ADD VAL:-0.79 idx:26/24  col:2
ADD VAL:-0.70 idx:30/25  col:2
ADD VAL:0.05 idx:3/27  col:3
ADD VAL:0.21 idx:7/28  col:3
ADD VAL:-1.95 idx:11/29  col:3
ADD VAL:0.64 idx:15/30  col:3
ADD VAL:1.14 idx:19/31  col:3
ADD VAL:-0.21 idx:23/32  col:3
ADD VAL:-0.13 idx:27/33  col:3
ADD VAL:1.14 idx:31/34  col:3

In [165]:
matadd[0][0]

0.13292159

In [166]:
datat_res = np.empty_like(adatat)
colst_res = np.empty_like(acolst)
nnzst_res = np.empty_like(annzt)
cl.enqueue_copy(queue, datat_res, adatat_buf)
cl.enqueue_copy(queue, colst_res, acolst_buf)
cl.enqueue_copy(queue, nnzst_res, annzst_buf)

In [167]:
adenseaddt = to_dense(datat_res, colst_res, nnzst_res, ellwat, a.T.shape).T
adenseaddt

array([[ 0.42808938,  0.13065036,  1.3358252 ,  1.02301455,  1.04371226,
         1.63082707,  0.95310652, -0.21064782],
       [ 2.42373562,  1.44250965,  0.90111405,  1.28772759,  1.89654911,
        -1.47844338, -1.74050379,  1.20511603],
       [ 0.60596263,  0.09997106,  0.93567544, -1.124542  ,  0.1703828 ,
        -1.30114567, -0.40459514, -0.67618853],
       [ 0.80504   ,  0.3377057 , -1.91360486,  1.01154184,  1.33087492,
        -0.13255358,  0.55492139,  1.74664724]])

In [168]:
a

array([[0.29516777, 0.33338296, 0.2910005 , 0.5361629 , 0.31872442,
        0.00900633, 0.53607106, 0.9474776 ],
       [0.6204036 , 0.21273285, 0.59999454, 0.7183614 , 0.87148714,
        0.69184786, 0.04276368, 0.46898165],
       [0.02936536, 0.8756672 , 0.41719204, 0.13531114, 0.09763599,
        0.21822612, 0.38778782, 0.0189462 ],
       [0.7515835 , 0.12442248, 0.0349297 , 0.37084636, 0.19291036,
        0.0798979 , 0.6807204 , 0.61051154]], dtype=float32)

In [169]:
matadd

array([[ 0.13292159, -0.20273261,  1.0448247 ,  0.4868517 ,  0.7249878 ,
         1.6218207 ,  0.41703546, -1.1581254 ],
       [ 1.8033321 ,  1.2297767 ,  0.3011195 ,  0.5693663 ,  1.025062  ,
        -2.1702912 , -1.7832675 ,  0.7361344 ],
       [ 0.5765973 , -0.77569616,  0.5184834 , -1.2598531 ,  0.07274681,
        -1.5193717 , -0.79238296, -0.6951347 ],
       [ 0.05345648,  0.21328321, -1.9485346 ,  0.6406955 ,  1.1379646 ,
        -0.21245147, -0.12579903,  1.1361357 ]], dtype=float32)

In [170]:
a_added

array([[ 0.42808938,  0.13065036,  1.3358252 ,  1.0230145 ,  1.0437123 ,
         1.6308271 ,  0.9531065 , -0.21064782],
       [ 2.4237356 ,  1.4425097 ,  0.90111405,  1.2877276 ,  1.8965491 ,
        -1.4784434 , -1.7405038 ,  1.205116  ],
       [ 0.60596263,  0.09997106,  0.93567544, -1.124542  ,  0.1703828 ,
        -1.3011457 , -0.40459514, -0.6761885 ],
       [ 0.80504   ,  0.3377057 , -1.9136049 ,  1.0115418 ,  1.3308749 ,
        -0.13255358,  0.5549214 ,  1.7466472 ]], dtype=float32)

In [171]:
adenseaddt == a_added

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

In [172]:
adenseaddt == a_added

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

### Make Random

In [180]:
rand = SparseTensor.uniform(2,4)
rand

<SparseTensor <GPUBuffer with shape (8,)> with grad None>

In [181]:
rand.to_numpy()

array([[0.36719346, 0.0992442 , 0.28531802, 0.        ],
       [0.44079751, 0.22904435, 0.        , 0.32733995]])

In [ ]:
rand.data

### update vals

In [ ]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)

In [ ]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint col = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint row = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [ ]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,bs], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa), x_cp_buf, x_idx_buf, y_idx_buf)

resa = np.empty_like(adata)
resaidx = np.zeros(acols.shape).astype(np.uint32)
resannz = np.zeros(annz.shape).astype(np.uint32)

cl.enqueue_copy(queue, resa, adata_buf)
cl.enqueue_copy(queue, resaidx, acols_buf)
cl.enqueue_copy(queue, resannz, annzs_buf)

In [ ]:
resa.shape, resaidx.shape, resannz.shape, ellwa, a.T.shape

In [ ]:
adenseadd = to_dense(resa, resaidx, resannz, ellwa, a.T.shape)
adenseadd

In [ ]:
adenseadd - adense

In [ ]:
adenseadd == adense

In [ ]:
ellwa

In [ ]:
adata2 = adata.reshape(-1, ellwa)
adata2

In [ ]:
resa = resa.reshape(-1, ellwa)
resa

In [ ]:
resa - adata2

In [ ]:
acols

In [ ]:
resaidx

In [ ]:
resannz

In [ ]:
annz

### update vals2

In [ ]:
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)

In [ ]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint row = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint col = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [ ]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,bs], None, adatat_buf, acolst_buf, annzst_buf, np.float32(1), np.uint32(ellwat), x_cp_buf, x_idx_buf, y_idx_buf)

resat = np.empty_like(adatat)
resaidxt = np.zeros(acolst.shape).astype(np.uint32)
resannzt = np.zeros(annzt.shape).astype(np.uint32)

cl.enqueue_copy(queue, resat, adatat_buf)
cl.enqueue_copy(queue, resaidxt, acolst_buf)
cl.enqueue_copy(queue, resannzt, annzst_buf)

In [ ]:
ellwa

In [ ]:
resat.shape, resaidxt.shape, resannzt.shape

In [ ]:
adenseaddt = to_dense(resat, resaidxt, resannzt, ellwat, a.T.shape)
adenseaddt

In [ ]:
adenseadd == adenseaddt.T

In [ ]:
adata2t = adatat.reshape(-1, ellwat)
adata2t

In [ ]:
resat = resat.reshape(-1, ellwat)
resat

In [ ]:
resat - adata2t

In [ ]:
acols

In [ ]:
resaidx

In [ ]:
resannz

In [ ]:
annz

# OTHER

import numpy as np
import pyopencl as cl

mf = cl.mem_flags

dim = 16
topk = 4

x = np.random.rand(dim).astype(np.float32)
y = np.random.rand(dim).astype(np.float32)
x.shape,y.shape

dim1 = 4
dim2 = 8
dim3 = 1

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(dim2,dim3).flatten().astype(np.float32)

a.shape, b.shape

In [ ]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
val_out_buf = cl.Buffer(ctx, mf.READ_WRITE, 4*topk*topk)
x_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                         __global  float* y,    // INPUT
                         __global  float* xout,    // INPUT
                         uint topk,
                         __global  uint* xoutidx,    // INPUT
                         __global  uint* youtidx    // INPUT
                        ) { // LOCAL SHARED BUFFER
  uint gid = get_global_id(0);
  uint n = get_global_size(0);
  
  xout[gid] = x[gid];
  xoutidx[gid] = gid;
  youtidx[gid] = gid;
  
  float valx = x[gid];
  float valy = y[gid];
  uint posx = 0;
  uint posy = 0;
  for (uint i = 0; i < n; i++) {
    float tempval = x[i];
    float tempval2 = y[i];
    bool larger = tempval > valx;
    bool larger2 = tempval2 > valy;
      
    posx += (larger)?1:0;
    posy += (larger2)?1:0;
  }
  //printf("posx:%i", posx);
  if (posx < topk) {
    xoutidx[posx] = gid;
  }
  if (posy < topk) {
    youtidx[posy] = gid;
  }
  if (gid < topk) {
    uint i = gid;
    for (uint j=0; j<topk; j++) {
      xout[gid*topk+j] = x[xoutidx[gid]] * y[youtidx[j]];
    }
  }
}""").build()

In [ ]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
event = knl(queue, [dim,], None, x_buf, y_buf, val_out_buf, np.uint32(topk), x_idx_buf, y_idx_buf)

#event.wait()
val_out = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, val_out, val_out_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf, wait_for=[event])
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [ ]:
val_out

In [ ]:
resxidx

In [ ]:
resyidx

In [ ]:
asdf

In [ ]:
from __future__ import division

KERNEL_CODE = """
// Thread block size
#define BLOCK_SIZE %(block_size)d
// Matrix dimensions
// (chosen as multiples of the thread block size for simplicity)
#define WA %(w_a)d // Matrix A width
#define HA %(h_a)d // Matrix A height
#define WB %(w_b)d // Matrix B width
#define HB WA  // Matrix B height
#define WC WB  // Matrix C width
#define HC HA  // Matrix C height
/*
 * Copyright 1993-2009 NVIDIA Corporation.  All rights reserved.
 *
 * NVIDIA Corporation and its licensors retain all intellectual property and
 * proprietary rights in and to this software and related documentation.
 * Any use, reproduction, disclosure, or distribution of this software
 * and related documentation without an express license agreement from
 * NVIDIA Corporation is strictly prohibited.
 *
 * Please refer to the applicable NVIDIA end user license agreement (EULA)
 * associated with this source code for terms and conditions that govern
 * your use of this NVIDIA software.
 *
 */
/* Matrix multiplication: C = A * B.
 * Device code.
 */
#define AS(j, i) As[i + j * BLOCK_SIZE]
#define BS(j, i) Bs[i + j * BLOCK_SIZE]
////////////////////////////////////////////////////////////////////////////////
//! Matrix multiplication on the device: C = A * B
//! WA is A's width and WB is B's width
////////////////////////////////////////////////////////////////////////////////
__kernel __attribute__((reqd_work_group_size(16,16,1))) 
void
matrixMul( __global float* C, __global float* A, __global float* B)
{
    __local float As[BLOCK_SIZE*BLOCK_SIZE];
    __local float Bs[BLOCK_SIZE*BLOCK_SIZE];
    // Block index
    int bx = get_group_id(0);
    int by = get_group_id(1);
    // Thread index
    int tx = get_local_id(0);
    int ty = get_local_id(1);
    // Index of the first sub-matrix of A processed by the block
    int aBegin = WA * BLOCK_SIZE * by;
    // Index of the last sub-matrix of A processed by the block
    int aEnd   = aBegin + WA - 1;
    // Step size used to iterate through the sub-matrices of A
    int aStep  = BLOCK_SIZE;
    // Index of the first sub-matrix of B processed by the block
    int bBegin = BLOCK_SIZE * bx;
    // Step size used to iterate through the sub-matrices of B
    int bStep  = BLOCK_SIZE * WB;
    // Csub is used to store the element of the block sub-matrix
    // that is computed by the thread
    float Csub = 0.0f;
    // Loop over all the sub-matrices of A and B
    // required to compute the block sub-matrix
    for (int a = aBegin, b = bBegin;
             a <= aEnd;
             a += aStep, b += bStep) {
        // Load the matrices from device memory
        // to shared memory; each thread loads
        // one element of each matrix
        AS(ty, tx) = A[a + WA * ty + tx];
        BS(ty, tx) = B[b + WB * ty + tx];
        // Synchronize to make sure the matrices are loaded
        barrier(CLK_LOCAL_MEM_FENCE);
        // Multiply the two matrices together;
        // each thread computes one element
        // of the block sub-matrix
        for (int k = 0; k < BLOCK_SIZE; ++k)
            Csub += AS(ty, k) * BS(k, tx);
        // Synchronize to make sure that the preceding
        // computation is done before loading two new
        // sub-matrices of A and B in the next iteration
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    // Write the block sub-matrix to device memory;
    // each thread writes one element
    C[get_global_id(1) * get_global_size(0) + get_global_id(0)] = Csub;
}
"""
